## Chapter 7

In [64]:
import sys
sys.path.append("..")
from importlib import reload

In [65]:
import Ch7.utilities as ch7_utils
reload(ch7_utils)

<module 'Ch7.utilities' from '../Ch7/utilities.py'>

In [11]:
from sympy import *
import numpy as np
init_printing(use_latex='mathjax')
s, w = symbols('s w', real=True)

### Problem 7.1

Each of the four parts of this problem would follow the same 4 steps:

##### Step 1:
Identify available and unavailable subsystems and their associated matrices: $A_{uu}, A_{ua}, A_{au}, A_{aa}, B_{u}, B_{a}, C_a$. $C_a$ is the measurement matrix of the available subsystem, assumed to be nonsingular.
##### Step 2:
Compute matrices as a function of entries of $L$:
\begin{eqnarray}
F &=& A_{uu} - L C_a A_{au} \\
\bar {\bar G} &=& (A_{ua} - L C_a A_{aa}) C_a^{-1} \\
H &=& B_u - L C_a B_a.
\end{eqnarray}
These matrices define the dynamics for $\mathbf{z} \triangleq \mathbf{\hat x_u} - Ly$: $\mathbf{\dot z} = F \mathbf{\hat x_u} + {\bar {\bar G}} y + Hu$

##### Step 3:
Identify a design objective: i.e. poles for characteristic polynomial of $F$.

##### Step 4:
Use Bass-Gura formalism on the unavailable subsystem dynamics to solve for the gain matrix, $L$, that meets the design objective from *Step 3*.

To make the solution general, I didn't consider the specific cases of a), b), c), and d) of this problem.  Rather, I have drawn block diagrams for the cases of reduced-order observers of orders 3, 2, and 1.  Steps 1-4 above can be used on the two-cart system that is the subject of Problem 7.1 with the specific measurability conditions of each of part a), b), c), and d).  Since the problem isn't to _actually_ design reduced-order observers, but rather to discuss criteria for their design, I think this approach is okay. 

#### a)

With only one available measurement:
![prob7p1a](Problem7p1a.png)

#### b) and c)
With two available measurements:
![prob7p1bc](Problem7p1bc.png)

#### d)
With three available measurements:
![prob7p1d](Problem7p1d.png)

### Problem 7.2

This problem uses the basic ideas of the Bass-Gura formalism from Chapter 6.  The full observer is designed as: 

In [54]:
# using constants from Problem 3.6
m, M, l, g, k, R, r = .1, 1., 1., 9.8, 1., 100., .02
A = np.array([[0., 1., 0., 0.], 
              [0., -k**2 / (M*r**2*R), -(m*g)/M, 0.], 
              [0., 0., 0., 1.], 
              [0, k**2 / (M*r**2*R*l), ((M+m)*g)/(M*l), 0.]])
C = np.array([[1., 0., 0., 0.]])
B = np.array([[0.], [k/(M*R*r)], [0.], [-k/(M*R*r*l)]])
D = np.zeros((1, 1))
w0 = 5.
closedLoopPoles = np.roots(np.array([1.0, 2.613*w0, (2.+np.sqrt(2))*w0**2, 2.613*w0**3, w0**4]))
K = ch7_utils.obsBassGura(A, C, closedLoopPoles)
pprint('K = ')
pprint(K)

K = 
[[  -11.935     ] 
 [  394.51033906] 
 [ -452.00581633] 
 [-1993.61883169]]


The gain matrix for the full observer is $K = [-11.935, 394.510, -452.006, -1993.619]^T$.

### Problem 7.3

This problem is conceptually-similar to Problem 7.1 above, so I will follow the steps I outlined above; see `reducedOrderObserver` in "utilities.py" for the actual implementation.

In [55]:
m, M, l, g, k, R, r = .1, 1., 1., 9.8, 1., 100., .02
A = np.array([[0., 1., 0., 0.], 
              [0., -k**2 / (M*r**2*R), -(m*g)/M, 0.], 
              [0., 0., 0., 1.], 
              [0, k**2 / (M*r**2*R*l), ((M+m)*g)/(M*l), 0.]])
B = np.array([[0.],[k/(M*R*r)],[0.],[-k/(M*R*r*l)]])
# make sure to properly size the input matrices
A11 = np.array(A[0, 0]).reshape((1, 1))
A12 = np.array(A[0, 1:]).reshape((1, 3))
A21 = np.array(A[1:, 0]).reshape((3, 1))
A22 = np.array(A[1:, 1:]).reshape((3, 3))
B1 = np.array(B[0]).reshape((1, 1))
B2 = np.array(B[1:]).reshape((3, 1))
C1 = np.array([[1]]).reshape((1, 1))
desiredPoles = np.roots(np.array([1.0, 2.*5., 2.*5.**2, 5.**3]))
L, Gbb, H = ch7_utils.reducedOrderObserver(A11, A12, A21, A22, B1, B2, C1, desiredPoles)
pprint('L = ')
pprint(L)

L = 
 [[ -15.        ]
  [ -62.02040816]
 [-212.55102041]]


So, the desired third-order observer with the cart-position measured can be achieved with $L = [-15, -62.020, -212.551]^T$.

### Problem 7.4

#### a)

In [70]:
Kv, Lv, J, Km, Omm, Kdp = 94.3, 1.0, 7900., 8.46e6, 45.9, 6.33e-6
A = np.array([[0., 1., 0., 0.], [0., 0., 1., 0.], [0., -Km/J, -(Km/J)*(Omm*J/Km), (Km/J)], [0., 0., -Kv*Kdp*J, -Lv*Kv]])
B = np.array([[0.], [0.], [0.], [Kv]])
C = np.array([[1., 0., 0., 0.]])
w0 = 20.
closedLoopPoles = np.roots(np.array([1.0, 2.613*w0, (2.+np.sqrt(2.))*w0**2, 2.613*w0**3, w0**4]))
Ktilde = ch7_utils.obsBassGura(A, C, closedLoopPoles)
pprint('K[disturbance-free] = ')
pprint(Ktilde)

K[disturbance-free] = 
 [[-8.79400000e+01]
  [ 3.24568261e+03]
  [ 3.83776582e+05]
 [-5.37765422e+04]]


#### b)

![prob7p4b](Problem7p4b.png)

The exogenous vector is $\mathbf{x}_0 = [d_r, d_p, d_q]^T$.  Using the block diagram from Figure 4.4, the exogenous contribution to the system dynamics can be found to be as in the block below.  Part a) above gave the disturbance-free gain matrix, now we just need to apply steps from pg. 272.  _The method in the book doesn't work so well, so I'm omitting._

* Step 1: design a disturbance free observer (done in Part a) already)
* Step 2: Find matrix $V$ using Step 1.
* Step 3: Find $K_0$ so that the disturbance dynamics have the desired error decay rate.

In [72]:
D = np.zeros((1, 3))
E = np.array([[0., 0., 0.], [1., 0., 0.], [0., Km / J, 0.], [0., 0., Kv]])
w0 = 30.
desiredExoPoles = np.roots(np.array([1.0, 2.*w0, 2.*w0**2, w0**3]))
K = ch7_utils.obsBassGura(A, C, closedLoopPoles, D, E, desiredExoPoles)
pprint('K[full] = ')
pprint(K)

K[full] = 
 [[-8.79400000e+01]
  [ 3.24568261e+03]
  [ 3.83776582e+05]
  [-5.37765422e+04]
  [ 0.00000000e+00]
  [ 0.00000000e+00]
 [ 0.00000000e+00]]


_After going back and forth on the r/ControlTheory subreddit, I'm convinced that something is missing in Friedland's argument.  Or, more likely, I have constructed the problem incorrectly.  Either way, it is not worth the time to continue working on this.  For the time-being, just return 0's for the gains.  I'm leaving this note so I won't go back and not know what was going on._

### Problem 7.5

In [16]:
HoverJd, KQoverJd, KDoverJd, BoverJd = symbols('HoverJd KQoverJd KDoverJd BoverJd', real=True)
l1, l2, l3, l4 = symbols('l1 l2 l3 l4', real=True)
A = Matrix([[0., 0., 1., 0.],
            [0., 0., 0., 1.],
            [-KDoverJd, -KQoverJd, 0, -HoverJd],
            [KQoverJd, -KDoverJd, HoverJd, 0]])
B = Matrix([[0., 0.],
            [0., 0.],
            [1., 0.],
            [0., 1.]])
C = Matrix([[1., 0., 0., 0.],
            [0., 1., 0., 0.]])
L = Matrix([[l1, l2], [-l2, l1], [l3, l4], [-l4, l3]])
E = Matrix([[-1., 0.],
            [0., -1.],
            [0., 0.],
            [0., 0.]])
# the available subsystem observability is the 2x2 identity
Ca = eye(2)
Aaa = zeros(2)
Aau = Matrix([[A[0, 2], A[0, 3], E[0, 0], E[0, 1]], [A[1, 2], A[1, 3], E[1, 0], E[1, 1]]])
Aua = Matrix([[A[2, 0], A[2, 1]], [A[3, 0], A[3, 1]], [0., 0.], [0., 0.]])
Auu = Matrix([[A[2, 2], A[2, 3], E[2, 0], E[2, 1]], [A[3, 2], A[3, 3], E[3, 0], E[3, 1]], [0., 0., 0., 0.], [0., 0., 0., 0.]])
#pprint('A22 = ')
#pprint(Auu)
#pprint('A12 = ')
#pprint(Aau)
#pprint('C1 = ')
#pprint(Ca)
Ba = zeros(2)
Bu = Matrix([[1., 0.], [0., 1.], [0., 0.], [0., 0.]])
F = Matrix(Auu) - L * Matrix(Ca) * Matrix(Aau)
Gb = (Matrix(Aua) - L*Matrix(Ca)*Matrix(Aaa))*Matrix(Ca)**-1 + F*L
H = Matrix(Bu) - L*Matrix(Ca)*Matrix(Ba)
#pprint('F = ')
pprint(F)
#pprint('Gb = ')
#pprint(Gb)
#pprint('H = ')
#pprint(H)

⎡    -1.0⋅l₁       -HoverJd - 1.0⋅l₂  1.0⋅l₁   1.0⋅l₂⎤
⎢                                                    ⎥
⎢HoverJd + 1.0⋅l₂       -1.0⋅l₁       -1.0⋅l₂  1.0⋅l₁⎥
⎢                                                    ⎥
⎢    -1.0⋅l₃            -1.0⋅l₄       1.0⋅l₃   1.0⋅l₄⎥
⎢                                                    ⎥
⎣     1.0⋅l₄            -1.0⋅l₃       -1.0⋅l₄  1.0⋅l₃⎦


The problem says that I should see only four unique elements in the matrix $F$.  The result below is a little confusing then...

#### a)

In [206]:
pprint('F = ')
pprint(F)
pprint('Gb = ')
pprint(Gb)
pprint('H = ')
pprint(H)

F = 
⎡    -1.0⋅l₁       -HoverJd - 1.0⋅l₂  1.0⋅l₁   1.0⋅l₂⎤
⎢                                                    ⎥
⎢HoverJd + 1.0⋅l₂       -1.0⋅l₁       -1.0⋅l₂  1.0⋅l₁⎥
⎢                                                    ⎥
⎢    -1.0⋅l₃            -1.0⋅l₄       1.0⋅l₃   1.0⋅l₄⎥
⎢                                                    ⎥
⎣     1.0⋅l₄            -1.0⋅l₃       -1.0⋅l₄  1.0⋅l₃⎦
Gb = 
⎡                  2                                                          
⎢-KDoverJd - 1.0⋅l₁  + 1.0⋅l₁⋅l₃ - 1.0⋅l₂⋅l₄ - l₂⋅(-HoverJd - 1.0⋅l₂)  -KQover
⎢                                                                             
⎢                                                                             
⎢KQoverJd + 1.0⋅l₁⋅l₂ - 1.0⋅l₁⋅l₄ + l₁⋅(HoverJd + 1.0⋅l₂) - 1.0⋅l₂⋅l₃   -KDove
⎢                                                                             
⎢                                            2         2                      
⎢             -1.0⋅l₁⋅l₃ + 1.0⋅l₂⋅l₄ + 1.0⋅l₃  - 1.

#### b)

In [17]:
p = F.charpoly(s)
coeffs = p.all_coeffs()
pprint(coeffs)

⎡                                 2                          2                
⎣1.0, 2.0⋅l₁ - 2.0⋅l₃, 1.0⋅HoverJd  + 2.0⋅HoverJd⋅l₂ + 1.0⋅l₁  - 2.0⋅l₁⋅l₃ + 1

     2                     2         2               2                        
.0⋅l₂  - 2.0⋅l₂⋅l₄ + 1.0⋅l₃  + 1.0⋅l₄ , - 2.0⋅HoverJd ⋅l₃ + 2.0⋅HoverJd⋅l₁⋅l₄ 

                                2   2              2   2⎤
- 2.0⋅HoverJd⋅l₂⋅l₃, 1.0⋅HoverJd ⋅l₃  + 1.0⋅HoverJd ⋅l₄ ⎦


#### c)

I have put in as much time on this problem as I care to.  The characteristic equation above clearly leads to a nonlinear optimization problem for $l_1, l_2, l_3, $ and $l_4$.  I tried to solve for it below by changing my initial guess to get reasonably-small $f$ evaluation and I got a pretty good estimate for when the oscillatory modes are completely deadened, but the minimizer wouldn't converge for non-zero frequencies in the two oscillatory modes ($\omega = 3000$).  Changing the objective function below to some other convex objective did the trick.  I was unable to get a quick unit test implemented for this, so I am leaving as is. 

In [91]:
from scipy.optimize import minimize
HoverJd, KDoverJd, KQoverJd = 3000., 30., 60.
coeffs = [1.0,
          2.*(l1 - l3),
          HoverJd**2 + 2.0*HoverJd*l2 + l1**2 - 2.0*l1*l3 + l2**2 - 2.0*l2*l4 + 1.0*l3**2 + 1.0*l4**2,
          -2.0*HoverJd**2*l3 + 2.0*HoverJd*l1*l4 - 2.0*HoverJd*l2*l3,
          HoverJd**2*l3**2 + HoverJd**2*l4**2
         ]
#pprint(factor(coeffs))
A = np.array([[0., 0., 1., 0.],
             [0., 0., 0., 1.],
             [-KDoverJd, -KQoverJd, 0., -HoverJd],
             [KQoverJd, -KDoverJd, HoverJd, 0.]])
B = np.array([[0., 0.],
            [0., 0.],
            [1., 0.],
            [0., 1.]])
C = np.array([[1., 0., 0., 0.],
            [0., 1., 0., 0.]])
E = np.array([[-1., 0.],
              [0., -1.],
              [0., 0.],
              [0., 0.]])
Ca = np.eye(2)
Aaa = np.zeros((2, 2))
Aau = np.array([[A[0, 2], A[0, 3], E[0, 0], E[0, 1]], [A[1, 2], A[1, 3], E[1, 0], E[1, 1]]])
Aua = np.array([[A[2, 0], A[2, 1]], [A[3, 0], A[3, 1]], [0., 0.], [0., 0.]])
Auu = np.array([[A[2, 2], A[2, 3], E[2, 0], E[2, 1]], [A[3, 2], A[3, 3], E[3, 0], E[3, 1]], [0., 0., 0., 0.], [0., 0., 0., 0.]])
openLoopCoeffs = np.array(Auu.charpoly(s).all_coeffs())
#pprint(np.roots(openLoopCoeffs))
# keep imaginary parts as-is and shoot for real part = -1000.
#desiredCoeffs = np.poly(np.array([np.complex(-1000., 3000.), np.complex(-1000., -3000.), np.complex(-1000., 0.), np.complex(-1000., 0.)]))
r = -1000.
i = 3000.
desiredCoeffs = np.poly(np.array([np.complex(r, 0.), np.complex(r, 0.), np.complex(r, i), np.complex(r, -i)]))
pprint(desiredCoeffs)
# setup objective for the optimization - x, y, z -> l2, l3, l4
def obj(w):
    x, y, z = w
    u = desiredCoeffs[1] / 2. - y
    f2 = (u - y)**2 + (x - z)**2 + HoverJd**2 + 2.*HoverJd*x - desiredCoeffs[2]
    f3 = 2. * HoverJd * (u*z - x*y - HoverJd*y) - desiredCoeffs[3]
    f4 = HoverJd**2*(y**2 + z**2) - desiredCoeffs[4]
    return (f2 + f3 + f4)**2 # this objective works! the more intuitive (f2**2 + f3**2 + f4**2) does not!

def grad(w):
    h = 1e-8
    w1 = np.array([w[0] + h, w[1], w[2], w[3]])
    g1 = (obj(w1) - obj(w)) / h
    w2 = np.array([w[0], w[1]+h, w[2], w[3]])
    g2 = (obj(w2) - obj(w)) / h
    w3 = np.array([w[0], w[1], w[2]+h, w[3]])
    g3 = (obj(w3) - obj(w)) / h
    return np.array([g1, g2, g3])

x0 = np.array([-3000., -.004, 1000.])
sol = minimize(obj, x0=x0, method='nelder-mead', options={'maxiter': 10000})
l = list(sol.x)
#pprint(l)
l.insert(0, desiredCoeffs[1] / 2. - sol.x[1]) # l1 as a function of l1 came from first coefficient
#pprint(obj(sol.x)) # check on how close the solution found gets to actual solution (f(x) = 0), curr ~ 10e-5, not bad
pprint('l1, l2, l3, l4 = {}'.format(l))
L = np.array([[l[0], l[1]], [-l[1], l[0]], [l[2], l[3]], [-l[3], l[2]]])
F = Auu - L.dot(Ca.dot(Aau))
pprint(np.poly(np.linalg.eigvals(F))) # this doesn't match super closely, but oh well - probably a combination of user-error and numerical instability of the minimizer (shrug)

[1.0e+00 4.0e+03 1.5e+07 2.2e+10 1.0e+13]
l1, l2, l3, l4 = [2000.0039425825403, -3113.2106773237365, -0.0039425825401703
08, 1054.5851351060987]
[1.00000000e+00 4.00001577e+03 1.16912894e+07 1.26550439e+10
                       1.00093483e+13]                      


### Problem 7.6

#### a)

In [68]:
R, _C = 2., 1.
# x == (v1, v2, v3)
A = np.array([[-3./(R*_C), 1./(R*_C), 0.],[1./(R*_C), -2./(R*_C), 1./(R*_C)], [0., 1./(R*_C), -3./(R*_C)]])
B = np.array([[2. / (R*_C)], [0.], [0.]])
C = np.array([[0., 0., 1.]])
w0 = 3.
desiredPoles = np.roots(np.array([1.0, 2.*w0, 2.*w0**2, w0**3]))
K = ch7_utils.obsBassGura(A, C, desiredPoles)
pprint('K[disturbance-free] = ')
pprint(K)

K[disturbance-free] = 
[[42.5] 
 [16.5] 
 [ 2. ]]


#### b)

I am not drawing any more block diagrams!!

In [69]:
D = np.zeros((1, 2))
E = np.array([[0., 0.], [0., 1./(R*_C)], [-3./(R*_C), 1./(R*_C)]])
desiredExoPoles = np.array([np.complex(-5., 5), np.complex(-5., -5)])
K = ch7_utils.obsBassGura(A, C, desiredPoles, D, E, desiredExoPoles)
pprint('K[full] = ')  # currently broken
pprint(K)

K[full] = 
[[42.5] 
 [16.5] 
 [ 2. ] 
 [ 0. ] 
 [ 0. ]]


#### c)

Took a stab at this one and it didn't work out as expected.  Considering that a valid observer was found with the same desired characteristics in part b), I'm not going to work through this any further.  _The metasystem is unobservable, so I don't know how this is expected to work.  I might have setup the problem incorrectly._

In [73]:
# XXX doesn't work
#R, C = 2., 1.
# x == (v1, v2, v3)
#A = np.array([[-3./(R*C), 1./(R*C), 0.],[1./(R*C), -2./(R*C), 1./(R*C)], [0., 1./(R*C), -3./(R*C)]])
#B = np.array([[2. / (R*C)], [0.], [0.]])
#E = np.array([[0., 0.], [0., 1./(R*C)], [-3. / (R*C), 1./(R*C)]])
#Am = np.zeros((5, 5))
#Am[:3, :3] = A
#Am[:3, 3:] = E
#Bm = np.zeros((5, 1))
#Bm[:3] = B
#Cm = np.array([[0., 0., 1., 1., 1.]])
#desiredCoeffsNoDist = np.array([1.0, 2.*w0, 2.*w0**2, w0**3])
#xRoots = np.array(np.roots(desiredCoeffsNoDist))
#x0Roots = np.array([np.complex(-5., 5), np.complex(-5., -5)])
#roots = np.array([xRoots[0], xRoots[1], xRoots[2], x0Roots[0], x0Roots[1]])
#pprint(roots)
#desiredCoeffs = np.poly(roots)
#_, K = BassGuraObsGain(Am, Cm, desiredCoeffs)
#pprint('Full K = ')
#pprint(K)

### Problem 7.7

Most of this problem is worked out in Example 7C, so I will just complete it.

#### a)

Solve the linear system from 7C.3 plugging in desired $\{ \hat b_i \}$.  The coefficients come from the desired poles given in the problem construction.

In [20]:
A = np.array([[-30.3, 0., 0., 0.], [1.2e-4, -6.02, 0., 0.], [0., -3.77, 0., 0.], [0., -2.8, 0., 0.]])
C = np.array([[0., 0., -7.3, 0.], [0., 0., 0., -25.0]])
desiredCoeffs = np.poly(np.array([np.complex(-18., 0.), np.complex(-9., np.sqrt(3.)/2.), np.complex(-9., -np.sqrt(3.)/2.)]))
b = np.array([[desiredCoeffs[3]],
             [desiredCoeffs[2] - A[0, 0]*A[0, 1]],
             [36.],
             [desiredCoeffs[1] + A[0, 0] + A[0, 1]]])
P = np.array([[-A[1, 0]*A[3, 1], A[0, 0]*A[3, 1], 0., A[0, 0]*A[1, 1]], 
              [0., -A[3, 1], 0., -(A[0, 0] + A[1, 1])],
              [0., 0., 1., 0.],
              [0., 0., 0., 1.]
              ])
k = np.linalg.inv(P).dot(b)
# following example -
tildeK12 = k[0] / C[1, 3]
tildeK22 = k[1] / C[1, 3]
tildeK31 = k[2] / C[0, 2]
tildeK42 = k[3] / C[1, 3]
K = np.array([[0., tildeK12[0]],
             [0., tildeK22[0]],
             [tildeK31[0], 0.],
             [0., tildeK42[0]]
             ])
pprint('K = ')
pprint(K)

K = 
 [[ 0.00000000e+00  6.65430000e+05]
  [ 0.00000000e+00 -2.83894286e+00]
  [-4.93150685e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.28000000e-01]]


#### b)

In [21]:
E = np.array([[0., 0.], [0., 0.], [62.2, 5.76], [0., 5.12]])
Ac = A - K.dot(C)
V = np.linalg.inv(Ac).dot(E)
pprint('V = ')
pprint(V)

V = 
 [[ 0.00000000e+00 -8.43444295e+05]
  [ 0.00000000e+00  1.29875686e+00]
  [-1.72777778e+00 -2.96008704e-01]
 [ 0.00000000e+00 -1.53623144e+00]]


#### c)

I'm not drawing any more block diagrams.